<a href="https://colab.research.google.com/github/SaqlainHussainShah/SQLi-Detection-using-Machine-Learning/blob/master/SQLI_2_CNN_64_norm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import time
import pandas as pd
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
from tensorflow import keras

In [0]:
import pandas as pd
import os

In [0]:
import numpy as np

In [0]:
df=pd.read_csv('./sqli.csv',encoding='utf-16')

In [14]:
df.head()

,Sentence,Label
0,NaN,1
1,select * from users where id = '1' union sele...,1
2,select * from users where id = '1' union sele...,1
3,"select * from users where id = 1 or ""%?"" or 1...",1
4,admin' or '1' = '1'--,1


In [0]:
df.dropna(axis=0, inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33757 entries, 1 to 33760
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  33757 non-null  object
 1   Label     33757 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 791.2+ KB


In [0]:
values = df.values

In [0]:


# vectorization of data

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, max_features=4096, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()



In [19]:
posts.shape

(33757, 4096)

In [0]:
posts.shape=(33757,64,64,1)

In [21]:
posts.shape

(33757, 64, 64, 1)

In [0]:
posts = posts/np.max(posts)

In [0]:
X=posts

In [24]:
np.max(posts)

13

In [0]:
y=df['Label']

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:


import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier



Using TensorFlow backend.


In [0]:


model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



In [59]:


model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 64)        640       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 29, 29, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               3211392   
_________________________________________________________________
dense_9 (Dense)              (None, 64)               

In [60]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=64)

Epoch 1/10
422/422 [==============================] - 4s 11ms/step - loss: 0.1950 - accuracy: 0.9345 - val_loss: 0.1464 - val_accuracy: 0.9588
Epoch 2/10
422/422 [==============================] - 4s 10ms/step - loss: 0.1334 - accuracy: 0.9624 - val_loss: 0.1242 - val_accuracy: 0.9652
Epoch 3/10
422/422 [==============================] - 4s 10ms/step - loss: 0.1206 - accuracy: 0.9663 - val_loss: 0.1205 - val_accuracy: 0.9664
Epoch 4/10
422/422 [==============================] - 4s 10ms/step - loss: 0.1157 - accuracy: 0.9686 - val_loss: 0.1190 - val_accuracy: 0.9670
Epoch 5/10
422/422 [==============================] - 4s 10ms/step - loss: 0.1158 - accuracy: 0.9689 - val_loss: 0.1247 - val_accuracy: 0.9680
Epoch 6/10
422/422 [==============================] - 4s 10ms/step - loss: 0.1103 - accuracy: 0.9702 - val_loss: 0.1184 - val_accuracy: 0.9670
Epoch 7/10
422/422 [==============================] - 4s 10ms/step - loss: 0.1076 - accuracy: 0.9713 - val_loss: 0.1213 - val_accuracy: 0.9665

In [0]:
pred=model.predict(X_test)

In [0]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [0]:
from sklearn.metrics import accuracy_score

In [64]:
accuracy_score(y_test,pred)

0.9669727488151659

In [0]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [0]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [0]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [0]:

def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1.0:
            if pred == 1.0:
                true_positive += 1
            elif pred != 1.0:
                false_negative += 1

        elif true == 0.0:
            if pred == 0.0:
                true_negative += 1
            elif pred != 0.0:
                false_positive += 1
       
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)



In [0]:
accuracy,precision,recall=confusion_matrix(y_test,pred)

In [71]:
print(" For CNN 2 layer 64 batch size and normalization \n Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For CNN 2 layer 64 batch size and normalization 
 Accuracy : 0.9669727488151659 
 Precision : 0.9849269901083373 
 Recall : 0.9163014899211218
